In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
from PIL import Image

import os
import copy
import re
from tqdm import tqdm
import torchvision.transforms.functional as TF
import torchvision.transforms as TT
    

In [2]:
partition = pd.read_csv("/kaggle/input/partition/list_eval_partition_correct.txt", delim_whitespace=True, names = ["name1", "name2", "id", "status"])

In [3]:
partition.insert(4, "score", [1]*len(partition))

In [4]:
partition["status"].value_counts()

train    98832
val      48935
test     47773
Name: status, dtype: int64

In [5]:
train_df = partition[partition["status"]== "train"]
test_df = partition[partition["status"]== "test"]
val_df = partition[partition["status"]== "val"]

In [6]:
train_ids = list(train_df["id"].unique())
test_ids = list(test_df["id"].unique())
val_ids = list(val_df["id"].unique())


In [7]:
root = "/kaggle/input/customer2shopdataset/Customer2Shop/Customer2Shop/img/"
categories =  os.listdir(root)
anotations = []
#os.makedirs("/kaggle/working/images/")
for cate in categories:
    
    folders = os.listdir(f"{root}{cate}/")

    #print(folders)
    
   
    for folder in folders:
        ids = os.listdir(f"{root}{cate}/"+folder)
        for id in ids:
            if id not in train_ids: 
                continue
            
            
            '''
            #make positive pairs
            images = os.listdir(os.path.join(f"{root}{cate}/{folder}/", id))
            try:
                selected_images = random.sample(images, k = 2)
            
                name1 = id  + selected_images[0]
                name2 = id  + selected_images[1]
                anotations.append(f"{name1}\t{name2}\t{1}")
                #print(images)

                image = Image.open(os.path.join(f'{root}{cate}/{folder}/', id, selected_images[0]))
                image.save("/kaggle/working/images/" + name1, format = "JPEG")  
                image = Image.open(os.path.join(f'{root}{cate}/{folder}/', id, selected_images[1]))
                image.save("/kaggle/working/images/"+ name2, format = "JPEG") 
            except : 
                print(cate, folder, id)
            '''


            #make negative pairs
            images = os.listdir(os.path.join(f"{root}{cate}/{folder}/", id))
            other_ids = copy.deepcopy(ids)
            other_ids.remove(id)
            
            #score 0.75
            oke = False 
            while oke == False:
                selected_id = random.choice(other_ids)
                if selected_id in train_ids:
                    oke = True
            
            
            images_ = os.listdir(os.path.join(f"{root}{cate}/{folder}/", selected_id))
            selected_images = [random.choice(images), random.choice(images_)]
            name1 = os.path.join(f'img/{cate}/{folder}/', id, selected_images[0])
            name2 = os.path.join(f'img/{cate}/{folder}/', selected_id, selected_images[1])
            anotations.append([name1, name2, "train", 0.75])
            
            
            #image = Image.open(os.path.join(f'{root}{cate}/{folder}/', id, selected_images[0]))
            #image.save("/kaggle/working/images/" + name1, format = "JPEG")  
            #image = Image.open(os.path.join(f'{root}{cate}/{folder}/', selected_id, selected_images[1]))
            #image.save("/kaggle/working/images/"+ name2, format = "JPEG") 
            
            #score 0
            selected_cate = random.choice(categories)
            list_folders = os.listdir(f"{root}{selected_cate}/")
            if folder in list_folders:
                list_folders.remove(folder)
            selected_folder = random.choice(list_folders)
            
            oke = False 
            while oke == False:
                selected_id = random.choice(os.listdir(f"{root}{selected_cate}/"+selected_folder))
                if selected_id in train_ids:
                    oke = True
            
            images_ = os.listdir(os.path.join(f"{root}{selected_cate}/{selected_folder}/", selected_id))
            selected_images = [random.choice(images), random.choice(images_)]
            
            name1 = os.path.join(f'img/{cate}/{folder}/', id, selected_images[0])
            name2 = os.path.join(f'img/{selected_cate}/{selected_folder}/', selected_id, selected_images[1])
            
            anotations.append([name1, name2, "train", 0])

            #image = Image.open(os.path.join(f'{root}{cate}/{folder}/', id, selected_images[0]))
            #image.save("/kaggle/working/images/" + name1, format = "JPEG")  
            #image = Image.open(os.path.join(f'{root}{cate}/{selected_folder}/', selected_id, selected_images[1]))
            #image.save("/kaggle/working/images/"+ name2, format = "JPEG") 
           
'''
with open("/kaggle/working/anotations.txt", "w") as file:
    file.write("\n".join(anotations))
'''

'\nwith open("/kaggle/working/anotations.txt", "w") as file:\n    file.write("\n".join(anotations))\n'

In [8]:
nega_train = pd.DataFrame(anotations, columns = ["name1", "name2", "status", "score"])

In [9]:
train = pd.concat([nega_train, train_df])

In [10]:
train

,name1,name2,status,score,id
0,img/DRESSES/Dress/id_00001223/comsumer_02.jpg,img/DRESSES/Dress/id_00001458/comsumer_02.jpg,train,0.75,NaN
1,img/DRESSES/Dress/id_00001223/comsumer_02.jpg,img/DRESSES/Skirt/id_00018464/comsumer_02.jpg,train,0.00,NaN
2,img/DRESSES/Dress/id_00000556/comsumer_03.jpg,img/DRESSES/Dress/id_00023699/comsumer_02.jpg,train,0.75,NaN
3,img/DRESSES/Dress/id_00000556/shop_01.jpg,img/CLOTHING/T_Shirt/id_00008194/comsumer_05.jpg,train,0.00,NaN
4,img/DRESSES/Dress/id_00025664/comsumer_01.jpg,img/DRESSES/Dress/id_00000752/comsumer_14.jpg,train,0.75,NaN
...,...,...,...,...,...
195519,img/DRESSES/Dress/id_00026698/comsumer_01.jpg,img/DRESSES/Dress/id_00026698/shop_01.jpg,train,1.00,id_00026698
195520,img/DRESSES/Dress/id_00026698/comsumer_02.jpg,img/DRESSES/Dress/id_00026698/shop_01.jpg,train,1.00,id_00026698
195521,img/DRESSES/Dress/id_00026698/comsumer_03.jpg,img/DRESSES/Dress/id_00026698/shop_01.jpg,train,1.00,id_00026698
195538,img/DRESSES/Dress/id_00026719/comsumer_01.jpg,img/DRESSES/Dress/id_00026719/shop_01.jpg,train,1.00,id_00026719


In [11]:
#make dir for output

root = "/kaggle/input/customer2shopdataset/Customer2Shop/Customer2Shop/img/"
categories =  os.listdir(root)
for cate in categories:
    folders = os.listdir(f"{root}{cate}/")
    for folder in folders:
        ids = os.listdir(f"{root}{cate}/"+folder)
        for id in ids:
            os.makedirs(os.path.join("img", cate, folder, id))
    


In [12]:
in_dir = "/kaggle/input/customer2shopdataset/Customer2Shop/Customer2Shop"
aug = []

def createImage(name1, name2, in_dir):
    img1 = Image.open(os.path.join(in_dir, name1))
    img2 = Image.open(os.path.join(in_dir, name2))
    
    img1.save(name1, format = "JPEG")
    img2.save(name2, format = "JPEG")
    
    x = random.randint(0, 4)
    
    match x:
        case 0:
            name_ = name1.replace(".jpg", "_rotate30.jpg")
            im_ = TF.rotate(img1, 30)
        case 1:
            name_ = name1.replace(".jpg", "_rotate180.jpg")
            im_ = TF.rotate(img1, 180)
        case 2:
            name_ = name1.replace(".jpg", "_flip.jpg")
            im_ = TF.hflip(img1)          
        case 3:
            name_ = name1.replace(".jpg", "_blur.jpg")
            im_ = TF.gaussian_blur(img1, 17)
        case 4:
            
            name_ = name1.replace(".jpg", "_crop.jpg")
            w, h = img1.size
            cropted_w, cropted_h = int(w*0.8), int(h*0.8)
            transform = TT.RandomCrop((cropted_h, cropted_w))
            im_ = transform(img1)
    
    im_.save(name_, format = "JPEG")
    aug.append([name_, name2, id,status, score])
    
for idx, row in tqdm(enumerate(train.itertuples())):
    name1 = row[1]
    name2 = row[2]
    id = row[3]
    status = row[4]
    score = row[5]
    createImage(name1, name2, in_dir)


134304it [1:01:40, 36.29it/s]


In [13]:
aug_df = pd.DataFrame(aug, columns = ["name1", "name2","id", "status", "score"])

In [14]:
final_train = pd.concat([train, aug_df])
final_train.to_csv("pairs_for_training.csv")

In [15]:
"""
root = "/kaggle/input/customer2shopdataset/Customer2Shop/Customer2Shop"
count = 0
for idx, row in tqdm(enumerate(test.itertuples())):
    name1 = row[1]
    name2 = row[2]
    score = row

    
    if (os.path.exists(os.path.join(root, name1)) == False) or (os.path.exists(os.path.join(root, name2)) == False):
        print(idx, name1, name2)
                
print(count)
"""


    


'\nroot = "/kaggle/input/customer2shopdataset/Customer2Shop/Customer2Shop"\ncount = 0\nfor idx, row in tqdm(enumerate(test.itertuples())):\n    name1 = row[1]\n    name2 = row[2]\n    score = row\n\n    \n    if (os.path.exists(os.path.join(root, name1)) == False) or (os.path.exists(os.path.join(root, name2)) == False):\n        print(idx, name1, name2)\n                \nprint(count)\n'

In [16]:
test =  pd.concat([test_df, val_df])

In [17]:
test

,name1,name2,id,status,score
0,img/TOPS/T_Shirt/id_00000001/comsumer_01.jpg,img/TOPS/T_Shirt/id_00000001/shop_02.jpg,id_00000001,test,1
1,img/TOPS/T_Shirt/id_00000001/comsumer_02.jpg,img/TOPS/T_Shirt/id_00000001/shop_01.jpg,id_00000001,test,1
2,img/TOPS/T_Shirt/id_00000001/comsumer_03.jpg,img/TOPS/T_Shirt/id_00000001/shop_01.jpg,id_00000001,test,1
3,img/TOPS/T_Shirt/id_00000001/comsumer_04.jpg,img/TOPS/T_Shirt/id_00000001/shop_03.jpg,id_00000001,test,1
4,img/TOPS/T_Shirt/id_00000001/comsumer_05.jpg,img/TOPS/T_Shirt/id_00000001/shop_01.jpg,id_00000001,test,1
...,...,...,...,...,...
195533,img/DRESSES/Dress/id_00026713/comsumer_10.jpg,img/DRESSES/Dress/id_00026713/shop_01.jpg,id_00026713,val,1
195534,img/DRESSES/Dress/id_00026713/comsumer_11.jpg,img/DRESSES/Dress/id_00026713/shop_01.jpg,id_00026713,val,1
195535,img/DRESSES/Dress/id_00026713/comsumer_12.jpg,img/DRESSES/Dress/id_00026713/shop_01.jpg,id_00026713,val,1
195536,img/DRESSES/Dress/id_00026713/comsumer_13.jpg,img/DRESSES/Dress/id_00026713/shop_01.jpg,id_00026713,val,1


In [18]:
test.to_csv("pairs_for_testing.csv")

In [19]:
for idx, row in tqdm(enumerate(test.itertuples())):
    name1 = row[1]
    name2 = row[2]
    img1 = Image.open(os.path.join(in_dir, name1))
    img2 = Image.open(os.path.join(in_dir, name2))
    
    img1.save(name1, format = "JPEG")
    img2.save(name2, format = "JPEG")
    


96708it [32:28, 49.63it/s]
